In [41]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [42]:
filepath = tf.keras.utils.get_file('shakespeare.txt', './data/shakespeare.txt')

In [43]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [44]:
text = text[100000:1000000]

In [45]:
characters = sorted(set(text))

In [46]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [112]:
SEQ_LENGTH = 64
STEP_SIZE = 8

In [113]:
sentences = []
next_characters = []

In [114]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [115]:
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
Y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [116]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    X[i, t, char_to_index[character]] = 1
  Y[i, char_to_index[next_characters[i]]] = 1

In [117]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
csv_logger = tf.keras.callbacks.CSVLogger('./training_log/log.csv', separator=',', append=True)

model.fit(X, Y, batch_size=256, epochs=10, callbacks=[csv_logger])

Epoch 1/10
440/440 [==============================] - 47s 102ms/step - loss: 2.2119
Epoch 2/10
440/440 [==============================] - 41s 94ms/step - loss: 1.7740
Epoch 3/10
440/440 [==============================] - 45s 103ms/step - loss: 1.6290
Epoch 4/10
440/440 [==============================] - 45s 102ms/step - loss: 1.5469
Epoch 5/10
440/440 [==============================] - 43s 98ms/step - loss: 1.4918
Epoch 6/10
440/440 [==============================] - 47s 106ms/step - loss: 1.4475
Epoch 7/10
440/440 [==============================] - 48s 110ms/step - loss: 1.4135
Epoch 8/10
440/440 [==============================] - 46s 105ms/step - loss: 1.3838
Epoch 9/10
440/440 [==============================] - 45s 102ms/step - loss: 1.3605
Epoch 10/10
440/440 [==============================] - 44s 101ms/step - loss: 1.3420


In [118]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [119]:
model = tf.keras.models.load_model('textgenerator.model')

In [120]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [121]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    A = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      A[0, t, char_to_index[character]] = 1
    
    predictions = model.predict(A, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  
  return generated

In [122]:
print('-----------------------------0.2-------------------------------')
print(generate_text(256, 0.2))
print('-----------------------------0.4-------------------------------')
print(generate_text(256, 0.4))
print('-----------------------------0.6-------------------------------')
print(generate_text(256, 0.6))
print('-----------------------------0.8-------------------------------')
print(generate_text(256, 0.8))
print('------------------------------1--------------------------------')
print(generate_text(256, 1))

-----------------------------0.2-------------------------------
e rigour of the statute,
to make him an example. all hope is goness.

gloucester:
i will the comest with the comest to the wife.

gloucester:
i will the charch and his hearth the comest to the will.

gloucester:
ay, but in the sin the shall the willors,
and so a sir, and the come to the duke to the will.

clarence:
why
-----------------------------0.4-------------------------------
.

juliet:
hist! romeo, hist! o, for a falconer's voice,
to lure can with the royal not for the fire
to see the blasted with his her like, and sir,
i will the court in the his dishorsely heard
and she the come to the children, the fire
that i will the hands, the willowed with the comest to the trait
brow hold upon the 
-----------------------------0.6-------------------------------
not know,
which a dismiss'd offence would after gall;
and do him he they the my tent but one executio
where's the reason that i and good my leaved
a soul and subjects